<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">

# Derivatives Analytics with Python
**&mdash;Part IIIa: Market-Based Valuation (Ch.&nbsp;8&ndash;10)**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>

## Part IIIa &mdash; Market-Based Valuation

### Chapter 9 &mdash; General Model Framework

This notebook introduces the general market model framework used in Part III. It connects model primitives (state variables, risk-neutral dynamics, discounting) to valuation building blocks for bonds and European options.

- Specify the BCC framework with stochastic volatility, jumps, and stochastic rates.
- Value zero-coupon bonds in the `CIR` short-rate model and derive a term structure.
- Price European calls by the Lewis (2001) transform method for key special cases.

Reference scripts for the book are available under `x_store/dawp/python36`.


### 1. Environment check

We start by importing the core libraries for this class and by printing version
information to confirm that a modern Python environment is active.

In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for data and figure export

import math  # elementary math functions

import numpy as np  # numerical arrays
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

from numpy.fft import fft  # fast Fourier transform
from numpy.random import default_rng  # random number generator

from scipy.integrate import quad  # numerical integration
import scipy.optimize as sop  # numerical optimization

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string
print("pandas:", pd.__version__)  # pandas version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import (  # Jupyter helper
        set_matplotlib_formats,
    )
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline rendering


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")
# Allow importing companion modules from ../code.
CODE_DIR = Path('..') / 'code'  # companion module directory
sys.path.insert(0, str(CODE_DIR.resolve()))  # prefer local modules



### 2. BCC model framework

The Bakshi&mdash;Cao&mdash;Chen (BCC) framework combines three risk drivers: index risk, stochastic volatility, and stochastic short rates. A jump component can be included to capture heavy tails and short-term skew.


### 3. CIR discounting: term structure

The Cox&mdash;Ingersoll&mdash;Ross (CIR) short-rate model implies affine zero-coupon bond prices. The next code cell produces the term structure in discount factors and continuously compounded yields and exports Figure `dawp_pIIIa_fig03_cir_term_structure`.


In [ ]:
from pathlib import Path  # path handling for optional figure export

import numpy as np  # numerical arrays
import matplotlib.pyplot as plt  # plotting

from dawp_pIIIa_ch09_cir_zcb import cir_zcb_value, cir_zcb_yield  # analytics

r0, kappa_r, theta_r, sigma_r = 0.04, 0.30, 0.04, 0.10  # CIR parameters
maturities = np.linspace(0.25, 10.0, 40)  # maturity grid

prices = np.array([
    cir_zcb_value(r0, kappa_r, theta_r, sigma_r, T=float(T))
    for T in maturities
])  # discount factors

yields = np.array([
    cir_zcb_yield(r0, kappa_r, theta_r, sigma_r, T=float(T))
    for T in maturities
])  # yields

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(maturities, prices, lw=2.2)
ax[0].set_ylabel('ZCB value $B_0(T)$')

ax[1].plot(maturities, 100.0 * yields, lw=2.2, color='tab:green')
ax[1].set_xlabel('maturity $T$ (years)')
ax[1].set_ylabel('yield $y_0(T)$ in %')

fig.suptitle('CIR (1985) term structure: discount factors and yields')
fig.tight_layout()

maybe_save(fig, 'dawp_pIIIa_fig03_cir_term_structure')
plt.show()


### 4. European call valuation by transforms

For many market models, the characteristic function of the log-return is available in closed form. The Lewis (2001) formula turns such a characteristic function into a single Fourier integral for the European call value.

The next cell compares call values across strikes for important special cases within the BCC framework and exports Figure `dawp_pIIIa_fig04_bcc_special_cases`.


In [ ]:
import math  # elementary math functions

import numpy as np  # numerical arrays
import matplotlib.pyplot as plt  # plotting

from dawp_pIIIa_ch08_m76_pricing import m76_call_value_int  # Merton reference
from dawp_pIIIa_ch09_bcc_option_valuation import (
    bcc_call_value_int,
    bsm_call_value,
    cir_implied_short_rate,
    h93_call_value_int,
)

S0, T = 100.0, 1.0  # spot and maturity
strikes = np.arange(70.0, 131.0, 5.0)  # strike grid

r0, kappa_r, theta_r, sigma_r = 0.04, 0.30, 0.04, 0.10
r = cir_implied_short_rate(r0, kappa_r, theta_r, sigma_r, T=T)

kappa_v, theta_v, sigma_v, rho, v0 = 1.5, 0.02, 0.15, -0.7, 0.02
lamb, mu, delta = 0.25, -0.20, 0.10
sigma = math.sqrt(v0)  # BSM volatility proxy

bsm = np.array([bsm_call_value(S0, K, T, r, sigma) for K in strikes])
m76 = np.array([m76_call_value_int(S0, K, T, r, sigma, lamb, mu, delta)
                for K in strikes])
h93 = np.array([h93_call_value_int(S0, K, T, r, kappa_v, theta_v,
                                   sigma_v, rho, v0) for K in strikes])
bcc = np.array([bcc_call_value_int(S0, K, T, r, kappa_v, theta_v,
                                   sigma_v, rho, v0, lamb, mu, delta)
                for K in strikes])

fig, ax = plt.subplots()
ax.plot(strikes, bsm, lw=2.0, label='BSM (diffusion)')
ax.plot(strikes, m76, lw=2.0, ls='--', label='Merton (jumps)')
ax.plot(strikes, h93, lw=2.0, ls='-.', label='Heston (stoch. vol)')
ax.plot(strikes, bcc, lw=2.0, color='tab:red',
        label='Bates (stoch. vol + jumps)')

ax.set_xlabel('strike $K$')
ax.set_ylabel('call value')
ax.set_title('European call values: special cases within the BCC framework')
ax.legend(loc=0)

maybe_save(fig, 'dawp_pIIIa_fig04_bcc_special_cases')
plt.show()


### 5. Summary

The BCC framework provides a consistent setup for market-based valuation: it combines a short-rate model for discounting with equity dynamics that can match volatility smiles and term structures. Transform methods provide fast European option prices for calibration, while Monte Carlo simulation addresses the most general cases.


<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">
